In [1]:
#implementation of Chatbot

In [6]:
pip install nltk scikit-learn streamlit

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import nltk
import random
import os
import ssl
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


In [7]:
ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Abishek\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
intents = [
    {
      "tag": "greeting",
      "patterns": ["Hello", "Hi", "Hey", "Good morning", "Good evening"],
      "responses": ["Hello! How can I assist you today?", "Hi there! How can I help?"]
    },
    {
      "tag": "goodbye",
      "patterns": ["Bye", "Goodbye", "See you later", "Take care"],
      "responses": ["Goodbye! Have a great day!", "See you later!"]
    },
    {
      "tag": "thanks",
      "patterns": ["Thanks", "Thank you", "I appreciate it"],
      "responses": ["You're welcome!", "Glad I could help!"]
    },
    {
      "tag": "name",
      "patterns": ["What is your name?", "Who are you?"],
      "responses": ["I am your AI chatbot!", "You can call me ChatBot."]
    },
    {
      "tag": "weather",
      "patterns": ["What is the weather like?", "Tell me the weather", "Is it raining today?"],
      "responses": ["I can check the weather for you. Which city?"]
    },
    {
      "tag": "help",
      "patterns": ["I need help", "Can you assist me?", "Help me"],
      "responses": ["Sure! What do you need help with?"]
    },
    {
  "tag": "about",
  "patterns": ["Tell me about yourself", "What do you do?", "Who created you?", "What is your purpose?"],
  "responses": [
    "I am an AI chatbot designed to assist you with various tasks.",
    "I am a virtual assistant created to answer your questions and provide information.",
    "I was designed to help users with queries, automation, and general assistance."
  ]
},
    {
  "tag": "age",
  "patterns": ["How old are you?", "What is your age?", "When were you created?", "How long have you existed?"],
  "responses": [
    "I exist in the digital world, so I don't age like humans!",
    "I was created recently, but I keep evolving and learning!",
    "I don’t have an exact age, but I'm always up-to-date and improving!",
    "Time doesn't affect me, but I'm always here to assist you!"
  ]
},{
  "tag": "budget",
  "patterns": [
    "What is my budget?", 
    "Can you help me with budgeting?", 
    "How to manage my budget?", 
    "Give me some budgeting tips", 
    "I need help with financial planning"
  ],
  "responses": [
    "Sure! Are you looking for general budgeting tips or help with a specific budget?",
    "A good rule of thumb is the 50/30/20 rule: 50% for needs, 30% for wants, and 20% for savings.",
    "You can track your income and expenses using budgeting apps or spreadsheets.",
    "Would you like some recommendations for financial planning tools?",
    "Creating a budget involves listing your income sources and setting limits on your expenses."
  ]
},{
  "tag": "credit_score",
  "patterns": [
    "What is a credit score?",
    "How can I check my credit score?",
    "How do I improve my credit score?",
    "What affects my credit score?",
    "Why is my credit score important?"
  ],
  "responses": [
    "A credit score is a numerical representation of your creditworthiness, usually ranging from 300 to 850.",
    "You can check your credit score through credit bureaus like Experian, Equifax, or TransUnion.",
    "To improve your credit score, make payments on time, reduce outstanding debts, and avoid opening too many new accounts at once.",
    "Your credit score is affected by payment history, credit utilization, length of credit history, types of credit, and recent inquiries.",
    "A good credit score helps you get better loan rates, credit card approvals, and even rental agreements."
  ]
},
    {
      "tag": "joke",
      "patterns": ["Tell me a joke", "Make me laugh", "Say something funny"],
      "responses": ["Why don’t skeletons fight each other? They don’t have the guts!", "Why did the math book look sad? It had too many problems."]
    },
    {
      "tag": "news",
      "patterns": ["What's the latest news?", "Tell me current events", "Any updates in the world?"],
      "responses": ["Here are the latest headlines...", "Let me check the news for you!", "Breaking news: AI chatbots are getting smarter!"]
    },
    {
      "tag": "motivation",
      "patterns": ["Give me motivation", "I need inspiration", "Motivate me"],
      "responses": ["Believe in yourself! Every day is a new beginning.", "Success is not final, failure is not fatal. It is the courage to continue that counts."]
    },
    {
      "tag": "sports",
      "patterns": ["Give me sports updates", "Who won the match?", "Latest sports news"],
      "responses": ["The latest sports news is...", "Your favorite team just won!", "Let me check the sports updates for you."]
    },
    {
      "tag": "programming",
      "patterns": ["Tell me about Python", "How to learn Java?", "What is the best programming language?"],
      "responses": ["Python is great for AI and automation!", "Java is widely used for web and mobile development.", "The best programming language depends on your goal!"]
    },
    {
      "tag": "time",
      "patterns": ["What time is it?", "Tell me the current time", "What's the time now?"],
      "responses": ["It's currently 2:30 PM.", "Let me check the time for you!", "Time flies when you're chatting with me!"]
    },
    {
      "tag": "date",
      "patterns": ["What's today's date?", "Tell me the date", "Which day is it?"],
      "responses": ["Today is March 9, 2025.", "It's Sunday today!", "Let me check the date for you."]
    },
    {
      "tag": "holiday",
      "patterns": ["When is the next holiday?", "Tell me about upcoming holidays", "Is today a holiday?"],
      "responses": ["The next holiday is on April 14th!", "Let me check the holiday calendar for you.", "Today is not a holiday, but you can still take a break!"]
    },
    {
      "tag": "food",
      "patterns": ["Recommend me a dish", "What should I eat today?", "Suggest some good food"],
      "responses": ["How about trying some pasta?", "Pizza is always a great choice!", "A healthy salad would be perfect today."]
    },
    {
      "tag": "exercise",
      "patterns": ["Suggest a workout", "How to stay fit?", "Best exercises for beginners?"],
      "responses": ["Try doing some push-ups and squats!", "A daily walk of 30 minutes can keep you fit.", "Yoga is a great way to stay healthy."]
    },
    {
      "tag": "travel",
      "patterns": ["Where should I travel?", "Suggest a vacation spot", "Best travel destinations?"],
      "responses": ["How about visiting Bali?", "Paris is a great place to explore!", "If you love adventure, try the Himalayas!"]
    }
  


]



In [9]:
vectorizer = TfidfVectorizer()
clf = LogisticRegression(random_state=0, max_iter=10000)


tags = []
patterns = []
for intent in intents:
    for pattern in intent['patterns']:
        tags.append(intent['tag'])
        patterns.append(pattern)

In [10]:

x = vectorizer.fit_transform(patterns)
y = tags
clf.fit(x, y)

LogisticRegression(max_iter=10000, random_state=0)

In [11]:

def chatbot(input_text):
    input_text = vectorizer.transform([input_text])
    tag = clf.predict(input_text)[0]
    for intent in intents:
        if intent['tag'] == tag:
            response = random.choice(intent['responses'])
            return response

In [12]:

user_input = "What's your age?"
response = chatbot(user_input)
print(response)

I was created recently, but I keep evolving and learning!


In [13]:
user_input = "Hello"
response = chatbot(user_input)
print(response)


Hello! How can I assist you today?
